In [24]:
! git clone https://github.com/hyintell/BLOOM-fine-tuning.git
%cd BLOOM-fine-tuning
! pip install -r requirements.txt 

c:\Users\Pierre-Hadrien\NanoGPT\BLOOM-fine-tuning-main\BLOOM-fine-tuning


Cloning into 'BLOOM-fine-tuning'...


^C


# Libraries

In [1]:
import torch
import transformers
from transformers import BloomTokenizerFast, BloomForCausalLM, TrainingArguments

from datasets import load_dataset

from utils import ModifiedTrainer, tokenise_data, data_collator

from transformers import GPT2Tokenizer, GPT2LMHeadModel, BloomForCausalLM, BloomTokenizerFast
from mingpt.model import GPT
from mingpt.utils import set_seed
from mingpt.bpe import BPETokenizer
set_seed(3407)

 # Main

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device = torch.device("cpu")

In [3]:
model_name = "bloom-560m"
use_gpt = True
if use_gpt:
    model = GPT.from_pretrained('gpt2-xl')
else:
    model = BloomForCausalLM.from_pretrained(f"bigscience/{model_name}")

    
tokeniser = BloomTokenizerFast.from_pretrained(f"bigscience/{model_name}", add_prefix_space=True)

number of parameters: 1557.61M


In [4]:
# dataset = load_dataset('tatsu-lab/alpaca')
dataset = load_dataset("json", data_files="../nba_stats_with_text.json")
input_ids = tokenise_data(dataset, tokeniser)

model.to("cpu")

Found cached dataset json (C:/Users/Pierre-Hadrien/.cache/huggingface/datasets/json/default-a88fadb3b7372eba/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51)


  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 10/10 [00:00<00:00, 133.33it/s]


GPT(
  (transformer): ModuleDict(
    (wte): Embedding(50257, 1600)
    (wpe): Embedding(1024, 1600)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (ln_1): LayerNorm((1600,), eps=1e-05, elementwise_affine=True)
        (attn): CausalSelfAttention(
          (c_attn): Linear(in_features=1600, out_features=4800, bias=True)
          (c_proj): Linear(in_features=1600, out_features=1600, bias=True)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1600,), eps=1e-05, elementwise_affine=True)
        (mlp): ModuleDict(
          (c_fc): Linear(in_features=1600, out_features=6400, bias=True)
          (c_proj): Linear(in_features=6400, out_features=1600, bias=True)
          (act): NewGELU()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): Block(
        (ln_1): LayerNorm((1600,), eps=1e-05, elementwise_affine=True)
      

In [29]:
from dataclasses import dataclass, field
from typing import Optional

import torch
import tqdm
from transformers import Trainer

class ModifiedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        input_ids = inputs["input_ids"].to("cpu")
        attention_mask = torch.ones_like(input_ids).bool().to("cpu")
        labels = inputs["input_ids"].to("cpu")
        
        if use_gpt:
            return model(idx=input_ids).loss
        else :
            return model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels,
            ).loss
        


def data_collator(features: list) -> dict:
    return {"input_ids": torch.stack([torch.LongTensor(f) for f in features])}


def tokenise_data(dataset, tokenizer, max_seq_length=512):
    tokenised_list = []
    for elem in tqdm.tqdm(dataset["train"]):
        tokenised_list.append(
            tokenizer.encode(
                elem["text"],
                max_length=max_seq_length,
                padding="max_length",
                truncation=True,
            )
        )
    return tokenised_list


@dataclass
class ModelArguments:
    model_name_or_path: Optional[str] = field(default="bigscience/bloom-560m")


@dataclass
class DataArguments:
    data_name_or_path: str = field(
        default="tatsu-lab/alpaca", metadata={"help": "Path to the training data."}
    )


In [30]:
model.is_parallelizable = True
model.model_parallel = True

training_args = TrainingArguments(
    "output",
    fp16=False,
    gradient_accumulation_steps= 1,
    per_device_train_batch_size = 2,
    learning_rate = 2e-5,
    num_train_epochs=2,
    logging_steps=10,
)


trainer = ModifiedTrainer(
    model=model,
    train_dataset=input_ids,
    args=training_args,
    data_collator=data_collator,
)


In [31]:
trainer.train()

C:\Users\Pierre-Hadrien\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.7_qbz5n2kfra8p0\LocalCache\local-packages\Python37\site-packages\transformers\optimization.py:395: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


  0%|          | 0/10 [00:00<?, ?it/s]

IndexError: index out of range in self

In [ ]:
def generate(prompt='', num_samples=10, steps=20, do_sample=True):
    # Load the trained model
    model.eval()

    # Tokenize the input prompt into integer input sequence
    encoded_input = tokeniser(prompt, return_tensors='pt').to(device)
    x = encoded_input['input_ids']

    # We'll process all desired num_samples in a batch, so expand out the batch dim
    x = x.expand(num_samples, -1)

    # Forward the model `steps` times to get samples, in a batch
    with torch.no_grad():
        y = model.generate(
            x,
            max_length= 100,  # Set max_length based on input sequence length + steps
            do_sample=do_sample,
            top_k=40,
            pad_token_id=tokeniser.pad_token_id
        )

    for i in range(num_samples):
        out = tokeniser.decode(y[i].cpu().squeeze().tolist())
        print('-' * 80)
        print(out)


In [ ]:
generate(prompt='Generate specific player stats for player: Herbert Jones', num_samples=5, steps=30)

--------------------------------------------------------------------------------
Generate specific player stats for player: Herbert Jones, game:
player: Herbert Jones, player: player, stats: [player: player, player: player, player: player, player: player, player: player], score: 0, game: game, player: player, player: player.]
I think the player stats should now include the player, game, player, player, player, player, player, player, player, player, player, player, player, player
--------------------------------------------------------------------------------
Generate specific player stats for player: Herbert Jones

This code will save the player stats of player Herbert Jones, who is found in the player statistics of player Herbert Jones, and is also the player in the player of player Herbert Jones player statistics, and player and player in player player player player player player player player player player player player player player player player player player player player player

In [30]:
generate(prompt='Generate specific player stats for player: Carlik Jones', num_samples=5, steps=30)

--------------------------------------------------------------------------------
Generate specific player stats for player: Carlik Jones
    *    If player has a player statistics, player statistics are saved for the player.
    *
    * @param player The player data to store player stats
    */
    void setPlayerStats(Creature & player);

    /**
    * Changes player stats for player: Rengar Targaryen
    *    If player has a player statistics, player stats are saved for the player.
    *
    * @param player The player data to store player stats
   
--------------------------------------------------------------------------------
Generate specific player stats for player: Carlik Jones has an average of 0.5 possessions, 0 attempts, 0 steals, 0 field goals, and 0 total field goals for he has a field goal percentage of 1.6%. The field goal percentage for Carlik Jones is 0.6% and he makes 0 field goal attempts per game. Carlik Jones makes 0 three-pointers and 0 three-pointers for the entire